In [93]:
# install
#!pip install pandas

In [69]:
# import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [70]:
# Chargement des données

# chemin et nom du fichier 
fichier_fr = "small_vocab_fr"

# Lire les lignes du fichier texte dans une liste
with open(fichier_fr, "r") as fichier:
    lignes = fichier.readlines()
    
# Créer un DataFrame à partir de la liste de lignes
df_fr = pd.DataFrame({"Phrases": lignes})


# même méthode pour le df English
# chemin et nom du fichier 
fichier_en = "small_vocab_en"

# Lire les lignes du fichier texte dans une liste
with open(fichier_en, "r") as fichier:
    lignes = fichier.readlines()
    
# Créer un DataFrame à partir de la liste de lignes
df_en = pd.DataFrame({"Phrases": lignes})

# Afficher les premières lignes des DataFrame pour vérifier
print(df_fr.head())
print(df_en.head())


                                             Phrases
0  new jersey est parfois calme pendant l' automn...
1  les états-unis est généralement froid en juill...
2  california est généralement calme en mars , et...
3  les états-unis est parfois légère en juin , et...
4  votre moins aimé fruit est le raisin , mais mo...
                                             Phrases
0  new jersey is sometimes quiet during autumn , ...
1  the united states is usually chilly during jul...
2  california is usually quiet during march , and...
3  the united states is sometimes mild during jun...
4  your least liked fruit is the grape , but my l...


In [149]:
#### BAG OF WORDS ####

# import
from sklearn.feature_extraction.text import CountVectorizer

# Création d'une fonction pour créer le bag of words d'un dataframe
def BOW (dataframe):
    
    # Créez une instance CountVectorizer
    vectorizer = CountVectorizer( ) # paramètre par défaut car résulat bof avec count_vectorizer = CountVectorizer(analyzer="word", ngram_range=(1, 1), token_pattern=r"[^' ']+" )

    # Vectorizer le df ( crééer le bag of words)  soit calcul du nombre d'apparition de chaque mot dans la phrase de la colonne "Phrase"
    bow = vectorizer.fit_transform(dataframe["Phrases"])

    # Converti la matrice numpy en sortie du vectorize en df
    bow_df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names_out())
    
    # Renvoie le bow au format df en sortie de la fonction, et également le vectorizer car on va le réutiliser pour .transform lorsqu'on voudra 'bowiser' une nouvelle phrase d'apèrs le bow existant 
    return bow_df, vectorizer


# Création du bow du dataframe français et anglais
bow_fr, vectorizer_fr = BOW(df_fr)
bow_en, vectorizer_en = BOW(df_en)

# Print (bow_fr.head(),bow_en.head())

In [150]:
# Ajout d'une colonne langue et du tag en ou fr correspondant
bow_fr['langue']='fr'
bow_en['langue']='en'

# Afficher les premières lignes des DataFrame pour vérifier
bow_fr.head()
bow_en.head()

,am,and,animal,animals,apple,apples,april,are,aren,august,...,where,white,why,winter,wonderful,would,yellow,you,your,langue
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,en
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,en


In [151]:
# Concatenation des bow en un seul dataset
data = pd.concat([bow_fr, bow_en], axis=0)

# Remplacement des Nan (générés suite à la concaténation) en zéro
data = data.fillna(0)

# Comptez le nombre de NaN dans chaque colonne, puis sum supplémentaire pour avoir le nomre total de Nan
nan_count = data.isna().sum().sum()
print(nan_count)

0


In [152]:
# Création des jeux target et feature
X = data.drop(columns=["langue"])
y = data["langue"]

# Division des données en ensembles d'apprentissage et de test (80% pour l'apprentissage, 20% pour le test et un prélèvement random)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
### Entrainement d'un premier modèle KNN

# Import
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Créer un objet k-NN (k-Nearest Neighbors)
knn = KNeighborsClassifier(n_neighbors=3)  # valeur à ajuster pê

# Entraîner le modèle k-NN sur les données d'entraînement
knn.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = knn.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)

# Afficher la précision
print("Précision du modèle k-NN :", accuracy)


Précision du modèle k-NN : 1.0


In [154]:
### Générateur de phrase à détecter étape 1

# Constituer un réservoir de phrases utilisant les mots du bow, ce qui revient à concatener les df_fr et df_en (et pas les bow_fr et bow_en)
phrases_tank = pd.concat([df_fr, df_en], axis=0)
phrases_tank.head()


,Phrases
0,new jersey est parfois calme pendant l' automn...
1,les états-unis est généralement froid en juill...
2,"california est généralement calme en mars , et..."
3,"les états-unis est parfois légère en juin , et..."
4,"votre moins aimé fruit est le raisin , mais mo..."


In [157]:
### Générateur de phrase à détecter étape 2

# Prendre une phrase au hasard utilisant les mots du bow
phrase_random = phrases_tank['Phrases'].sample(n=1).iloc[0]
print("La phrase a détecter est :\n", phrase_random)

La phrase a détecter est :
 son fruit préféré est la mangue , mais notre préféré est la poire .



In [160]:
### Générateur de phrase à détecter étape 3 (optionnel)

# l'idée est de prendre X mots au hasard dans le bow commun
# même si aucun sens ou pas dans la même langue, juste pour voir quelle langue est atrribuée lors de la détection

import random

# Obtenir le vocabulaire commun à partir du vectorizer commun
bow_commun = vectorizer_commun.get_feature_names_out()

# Définir le nombre de mots que vous souhaitez dans la phrase au hasard (X)
X = 5  # Changez X selon les besoins

# Sélectionnez X mots au hasard à partir du vocabulaire commun
mots_au_hasard = random.sample(vocabulaire_commun, X)

# Construisez une phrase au hasard en utilisant les mots sélectionnés
phrase_au_hasard = " ".join(mots_au_hasard)

# Affichez la phrase au hasard
print("Phrase au hasard :", phrase_au_hasard)


In [159]:
# Prédire la langue de la phrase sélectionnée au hasard

    # étape 1 : vectoriser en bag of words la phrase sélectionnée
phrase_random_df = pd.DataFrame({'Phrases': phrase_random},index=[0]) # transfo de la phrase 'string' en dataframe car ma fonction BOW fonctionne avec des dataframes

#bow_phrase = BOW(phrase_random_df) ## hé non ! car cela créer un nouveau bow alors qu'il faut analyser la phrase avec le bow existant

    # Transformer la nouvelle phrase en vecteur BoW en utilisant le vectoriseur, pour "bowiser" avec le bow existant
bow_phrase = vectorizer.transform([phrase_random_df])

    # étape 2 : faire le prédict
langue_pred = knn.predict(bow_phrase)
print("La phrase suivant :\n", phrase_randomlangue_pred, "/n est en :", langue_pred)


## note pour moi : ça ne marche pas car je n'ai pas créer de vectorizer commun 

AttributeError: 'DataFrame' object has no attribute 'lower'

In [ ]:

# Créez une instance de SVM à noyau linéaire
#svm = SVC(kernel='linear', C=1.0)

# entrainement sur le train
#svm.fit(X_train, y_train)

# prediction
#svm.predict(X_test)

# score
#precision = svm.score(X_test, y_test)

#print (precision)

In [161]:
## etapes d'après :
# essayer avec des mots non contenus dans le bow et donc pouvoir saisir des phrases inventées non contenus dans les datasets


In [62]:
# Test saisie de texte
user_input = input("Entrez votre nom : ")
print("Bonjour, " + user_input + " !")

Entrez votre nom : Keyne
Bonjour, Keyne !
